In [ ]:
import numpy as np
import os
import shutil
from pImpactR import MLI as mli
from pImpactR import opt
from pImpactR.util import Me
from copy import deepcopy as copy
import time
import pickle
import matplotlib.pyplot as plt
# import psutil

In [ ]:
Espread = 2.0e-3
npt = 2048*2
nturn = 512
nturnSlice = 8
hole = 0.25
CLw = 1

# Read MLI input file

In [ ]:
elems0,lattices,labor = mli.readInputfile('mli.in.t3_iota_8_4_t0p4')
del(lattices[-1])

In [ ]:
elems0[2]

In [ ]:
energy=elems0[0].energy*1.0e9
gam0 = energy/Me
bet0 = np.sqrt(1.0-1.0/gam0**2)

In [ ]:
elems = []
for item in elems0:
    if not item.name in ['tasm','aim','vary','clear','anaprint']:
        elems.append(item)
# elems[2].driftexact = 1

In [ ]:
labor = ['iotaline','mapout','fin']

# Thick sext 2 Thin multipole

In [ ]:
elemList,latticeList=mli.sext2thin(elems,lattices,brho=0.50204778184582999)

In [ ]:
indexThin = []
k2lList = []
for i,item in enumerate(elemList):
    if 'thlm' == item.elem:
        indexThin.append(i)
        k2lList.append(item.k2l)
nThin = len(indexThin)
print(nThin)

# Define parameters, getInvTBT

In [ ]:
NL_nu = 0.3
NL_L  = 1.8
NL_c  = 0.01
NL_t  = 0.4
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx


f3 = k/(2*bet0)
f4 = k/(2*bet0*gam0)**2
print(bet0,k,f3,f4)

In [ ]:
emit = 3.3e-6
bg = gam0*bet0
sx = CLw*np.sqrt(betx*emit)
spx = CLw*np.sqrt((1+alfx*alfx)/betx*emit)
sy = CLw*np.sqrt(betx*emit)
spy = CLw*np.sqrt((1+alfx*alfx)/betx*emit)
se = CLw*Espread*bet0
print(sx,spx,se)

In [ ]:
def getTBT(npt,nturn,fname='rays.out'):
    TBT = np.loadtxt(fname)
    TBT = TBT[:npt*nturn,:6]
    out = np.zeros([npt,nturn,6])
    for i in range(nturn):
        out[:,i,:] = TBT[i*npt:(i+1)*npt,:]
        out[:,i,:] = MLI2norm(out[:,i,:])
    return out,TBT[npt*(nturn-1):,:]

In [ ]:
def getInv(xn,pxn,yn,pyn,delta,tau=NL_t):
    z = xn + 1j*yn
    U = np.real(z/np.sqrt(1-z**2)*np.arcsin(z))
    W = np.real(2*xn/np.sqrt(1-z**2)*np.arcsin(z))
    Hn = 0.5*(xn**2+pxn**2+yn**2+pyn**2)   +tau*U/(1.0+delta)
    In = (xn*pyn -yn*pxn)**2 +xn**2+pxn**2 +tau*W/(1.0+delta)
    return Hn,In

def getInvTBT(TBT):
    npt,nturn,dummy = TBT.shape
    InvTBT = np.zeros([npt,nturn,2])
    for iturn in range(nturn):
        data = TBT[:,iturn,:]
        for ipt in range(npt):
            xn    = data[ipt,0]
            pxn   = data[ipt,1]
            yn    = data[ipt,2]
            pyn   = data[ipt,3]
            delta = data[ipt,5]
            InvTBT[ipt,iturn,:] = getInv(xn,pxn,yn,pyn,delta)
    return InvTBT

#  Prepare particles

In [ ]:
def MLI2norm(data_in,sign=1):
    data=data_in.copy()
    data[:,5] = np.sqrt(1.0-2.0*data[:,5]/bet0+data[:,5]**2)-1.0
    data[:,1] = data[:,0]*alfx/np.sqrt(betx) + data[:,1]/(1+data[:,5])*np.sqrt(betx)
    data[:,3] = data[:,2]*alfx/np.sqrt(betx) + data[:,3]/(1+data[:,5])*np.sqrt(betx)
    data[:,0] = data[:,0]/(np.sqrt(betx))
    data[:,2] = data[:,2]/(np.sqrt(betx))
    return data
    
def norm2MLI(data_in,sign=1):
    data=data_in.copy()
    data[:,1] = (-data[:,0]*alfx*sign + data[:,1])/np.sqrt(betx)*(1+data[:,5])
    data[:,3] = (-data[:,2]*alfx*sign + data[:,3])/np.sqrt(betx)*(1+data[:,5])
    data[:,0] = data[:,0]*np.sqrt(betx)
    data[:,2] = data[:,2]*np.sqrt(betx)
    data[:,5] = -np.sqrt((1.0+data[:,5])**2+1.0/(bet0*gam0)**2)+1.0/bet0
    return data

In [ ]:
from scipy.stats import truncnorm

def get_truncated_normal(mean=0, sd=1.0, low=-3.0, upp=3.0, hole=0.0, n=1):
    x = np.zeros(n)
    for i in range(n):
        x[i] = 0.0
        while np.abs(x[i]) <= hole:
            f = truncnorm(
                (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)
            x[i] = f.rvs(1)
    return x

In [ ]:
x=get_truncated_normal(sd=1.0,low=-CLw,upp=CLw,n=npt*5,hole=hole)
pData=np.zeros([npt,6])
pData[:,[0,1,2,3,5]]=x.reshape([npt,5])
pData[:,:4] = pData[:,:4]*np.sqrt(emit)
pData[:,5] = pData[:,5]*Espread
pDataIn = norm2MLI(pData,sign=1)
np.savetxt('./origin/rays.in',pDataIn)

In [ ]:
plt.hist(x,bins=128);

# MLI Input for arc map

In [ ]:
ElemArcMap = elemList
mapout = ElemArcMap[-1]
mapout

In [ ]:
LineArcMap = latticeList[0]
LineArcMap.name = 'LineArcMap'
print(LineArcMap.list[0:8])
print(LineArcMap.list[-8:])

In [ ]:
for item in ElemArcMap:
    if item.name in ['nlr2','nlr1']:
        print(item)

In [ ]:
LaborArcMap = labor
LaborArcMap[0]='LineArcMap'
LaborArcMap

##### test

In [ ]:
mli.writeInputfile(ElemArcMap,[LineArcMap],LaborArcMap)
mli.run()
M_arc,G_arc=mli.readTransferMap()
M_arc

# MLI Input for one-turn map

In [ ]:
ElemOneturnMap = elemList[:3]
clear = mli.getElem.clear
readmap = mli.getElem.tmi(name='readmap')
savemap = mli.getElem.stm(name='savemap')
getmap  = mli.getElem.gtm(name='getmap')
getmap.iopt = 1 # concatenate
nlinsert = mli.getElem.nlinsert(steps=100)
# nlinsert = mli.getElem.drift(l=1.8)
ElemOneturnMap = ElemOneturnMap + [readmap,savemap,getmap,nlinsert,mapout,clear]
print(clear)
print(readmap)
print(savemap)
print(getmap)

In [ ]:
readmap.map2file(M_arc,G_arc)
! rm mli.out

In [ ]:
LineOneturnMap = mli.getElem.line(name='LineOneturnMap',elemList = [nlinsert,'getmap'])

In [ ]:
LaborOneturnMap = mli.buildLabor([readmap,savemap,clear,'LineOneturnMap','mapout','fin'])

##### test

In [ ]:
mli.writeInputfile(ElemOneturnMap,[LineOneturnMap],LaborOneturnMap)
mli.run()
M_oneturn,G_oneturn=mli.readTransferMap()
M_oneturn

# MLI Input for tracking

In [ ]:
ElemTrack = copy(ElemOneturnMap)
raysin  =mli.getElem.raytrace(file1='rays.in',type='readonly')
dump    =mli.getElem.particledump(file='rays.out',precision=15)
track   =mli.getElem.autotrack(type='symplectic',order=5)
ElemTrack = ElemTrack + [raysin,dump,track]
LineTrack = mli.getElem.line(name='LineTrack',elemList = [nlinsert,getmap,dump])
LaborTrack = mli.buildLabor([readmap,savemap,clear,raysin,track,str(nturnSlice)+'*'+LineTrack.name,'fin'])

##### test

In [ ]:
# np.savetxt('rays.in',pDataIn)

In [ ]:
# readmap.map2file(M_arc,G_arc)
# mli.writeInputfile(ElemTrack,[LineTrack],LaborTrack)
# mli.run()
# TBT,pDataOut = getTBT(128,nturnSlice)
# Inv0 = getInvTBT(TBT,nturnSlice)

In [ ]:
# readmap.map2file(M_oneturn,G_oneturn)
# LineTrack = mli.getElem.line(name='LineTrack',elemList = [getmap,dump.name])
# mli.writeInputfile(ElemTrack,[LineTrack],LaborTrack)
# mli.run()
# TBT,pDataOut = getTBT(128,nturnSlice)
# Inv1 = getInvTBT(TBT,nturnSlice)

In [ ]:
# plt.figure(figsize=(8,3))
# plt.subplot(1,2,1)
# for i in range(128):
#     plt.plot(Inv0[i,:,0])
# plt.subplot(1,2,2)
# for i in range(128):
#     plt.plot(Inv1[i,:,0])

# Optimize

### Build objective

In [ ]:
def getWeight(g):
    w = np.zeros(len(g))
    for i,item in enumerate(g['exponents']):
        nx = int(item[3])
        npx = int(item[4])
        ny = int(item[6])
        npy = int(item[7])
        nE = int(item[10])
        w[i] = np.power(sx,nx)*np.power(spx,npx)*np.power(sy,ny)*np.power(spy,npy)*np.power(se,nE)
    return w

In [ ]:
G2_ref = G_arc.loc[28:76].copy()
G2_ref['GP'] = 0
G2_ref.loc[33,'GP'] = 0.5*k/bet0
G2_ref.loc[67,'GP'] = 0.5*k/bet0
W2 = getWeight(G2_ref)
# G2_ref.head()

In [ ]:
G3_ref = G_arc.loc[84:200].copy()
G3_ref['GP'] = 0
G3_ref.loc[104,'GP'] = k/(2.0*bet0*gam0)**2
G3_ref.loc[184,'GP'] = k/(2.0*bet0*gam0)**2
W3 = getWeight(G3_ref)

In [ ]:
G4_ref = G_arc.loc[210:450].copy()
G4_ref['GP'] = 0
W4 = getWeight(G4_ref)

In [ ]:
G5_ref = G_arc.loc[462:910].copy()
G5_ref['GP'] = 0
W5 = getWeight(G5_ref)

In [ ]:
def getINVobj(INV,INV0):
    nturn = len(INV[0,:,0])
    obj=0.0
    tmpHmax = 0.0
    tmpImax = 0.0
    for i in range(nturn):
        tmpH = (INV[:,i,0]/INV0[:,0]-1.0)**2 
        tmpI = (INV[:,i,1]/INV0[:,1]-1.0)**2
        tmpHmax = tmpH.max()
        tmpImax = tmpI.max()
        tmp = (36.0*np.sum(tmpH)/npt + 9.0*np.sum(tmpI)/npt + 4.0*tmpHmax + tmpImax)/50.0
        obj = obj + tmp
        if tmp > 1.0:
            return obj/(i+1)
    return obj/nturn

In [ ]:
# pData=np.zeros([npt,6])
# x = np.random.random([npt,4])
# c0 = np.cos(x[:,0])
# s0 = np.sin(x[:,0])
# c1 = np.cos(x[:,1])
# s1 = np.sin(x[:,1])
# c2 = np.cos(x[:,2])
# s2 = np.sin(x[:,2])
# pData[:,0] = Rsphere*c0
# pData[:,1] = Rsphere*s0*c1
# pData[:,2] = Rsphere*s0*s1*c2
# pData[:,3] = Rsphere*s0*s1*s2
# pData[:,5] = (x[:,3]-0.5)*2*Espread
# pDataIn = norm2MLI(pData,sign=1)

In [ ]:
#%%
def objFunc(arg): 
    target = opt.id_generator()  # generage random directory name
    while os.path.exists(target):  
        target = opt.id_generator()
    shutil.copytree('origin', target)
    os.chdir(target) # cd to the randome directory and
    
    for i,j in enumerate(indexThin):
        elemList[j]['k2l']=arg[i]
    

    # 1st Objective : ArcMap
    mli.writeInputfile(ElemArcMap,[LineArcMap],LaborArcMap)
    mli.run()
    M,G = mli.readTransferMap()
    
    test = np.all(G2_ref['exponents'].values == G.loc[28 :76 ]['exponents'].values) and \
           np.all(G3_ref['exponents'].values == G.loc[84 :200]['exponents'].values) and \
           np.all(G4_ref['exponents'].values == G.loc[210:450]['exponents'].values) and \
           np.all(G5_ref['exponents'].values == G.loc[462:910]['exponents'].values)
    
    if not test:
        raise ValueError("PROBLEM!!!")
        
    obj = np.sum(((G.loc[28 :76 ,'GP'].values-G2_ref['GP'].values)*W2)**2) + \
          np.sum(((G.loc[84 :200,'GP'].values-G3_ref['GP'].values)*W3)**2) + \
          np.sum( (G.loc[210:450,'GP'].values*W4)**2) + \
          np.sum( (G.loc[462:910,'GP'].values*W5)**2)
    obj = obj*1.0e11
    obj = obj**0.2
    if obj > 1.0:
        os.chdir('..')
        shutil.rmtree(target)
        return obj
    
    # 2nd Objective : TBTInv
          
    readmap.map2file(M,G)
    mli.writeInputfile(ElemTrack,[LineTrack],LaborTrack)
    mli.run(4)       
    TBT,pDataOut = getTBT(npt,nturnSlice)
    !rm rays.out
    if(np.isnan(TBT.max())):
        os.chdir('..')
        shutil.rmtree(target)
        return 1.0
    Inv  = getInvTBT(TBT)
    Inv0 = Inv[:,0,:]
    obj2 = getINVobj(Inv,Inv0)
    
    if obj2 > 1.0:
        os.chdir('..')
        shutil.rmtree(target)
        return obj*obj2
    
    obj2tot = obj2
    for i in range(int(nturn/nturnSlice)-1):
        np.savetxt('rays.in',pDataOut)
        mli.run(4)
#         if i<48:
#             mli.run(8)
#         else:
#             mli.run(12)
#             nP = 0
#             for p in psutil.process_iter():
#                 if p.name()[:5] == 'mli.x':
#                     nP=nP+1
#             if nP>48:
#                 mli.run()
#             elif nP>32:
#                 mli.run(2)
#             elif np>24:
#                 mli.run(4)
#             elif np>16:
#                 mli.run(8)
#             elif np>8:
#                 mli.run(16)
#             else:
#                 mli.run(32)
        TBT,pDataOut = getTBT(npt,nturnSlice)
        !rm rays.out
        if(np.isnan(TBT.max())):
            os.chdir('..')
            shutil.rmtree(target)
            return obj*obj2
        
        Inv  = getInvTBT(TBT)
        Inv0 = Inv[:,0,:]
        obj2_tmp = getINVobj(Inv,Inv0)
        obj2tot = obj2tot+obj2_tmp
        obj2 = (obj2tot/(i+1))*(0.8**(i+1))
        
        if obj2_tmp > 1.0:
            os.chdir('..')
            shutil.rmtree(target)
            return obj*obj2
    os.chdir('..')
    shutil.rmtree(target)
    return obj*obj2

In [ ]:
with open('result.thin.3sig.sext','rb') as fp:
    result=pickle.load(fp)

In [ ]:
# objFunc(result.x)

In [ ]:
# objFunc([0]*nThin)

In [ ]:
# #%% run optim
bounds = [(-100,100)]*nThin
# result=opt.differential_evolution(objFunc, bounds, ncore=32, popsize=32*8,
#                                   disp=True, polish=True, maxtime=60*10) 
# #                                     stop running at maximum 10 min

In [ ]:
# with open('result.Inv.thin.'+str(CLw)+'sig.sext','wb') as fp:
#     pickle.dump(result,fp)

In [ ]:
with open('result.Inv.thin.'+str(CLw)+'sig.sext','rb') as fp:
    result=pickle.load(fp)

In [ ]:
result.message

In [ ]:
np.sort(result.population_energies)[:16]

In [ ]:
np.sort(result.population_energies)[-16:]

In [ ]:
while True:
    previous_result = result
    if hasattr(result,'x'): 
        break
    result = opt.differential_evolution(objFunc, bounds, ncore=4, 
                                           prev_result=previous_result, 
                                           disp=True, maxtime=60*10)
    with open('result.Inv.thin.'+str(CLw)+'sig.sext','wb') as fp:
        pickle.dump(result,fp)

In [ ]:
result